In [1]:
import pandas as pd
import numpy as np
import os
import openai

In [2]:
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
client = openai.OpenAI(api_key = api_key)

In [4]:
df = pd.read_csv("../movie.csv")

In [6]:
df.tail()

,text,label
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1
39999,"Why would this film be so good, but only gross...",1


In [7]:
df['label'].replace({0:'negative', 1:'positive'}, inplace=True)
df.head()

C:\Users\Bushika7\AppData\Local\Temp\ipykernel_8372\1593159159.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].replace({0:'negative', 1:'positive'}, inplace=True)


,text,label
0,I grew up (b. 1965) watching and loving the Th...,negative
1,"When I put this movie in my DVD player, and sa...",negative
2,Why do people who do not know what a particula...,negative
3,Even though I have great interest in Biblical ...,negative
4,Im a die hard Dads Army fan and nothing will e...,positive


In [52]:
system_message = '''
You are an AI assistant who's task is to do sentiment analysis on multiple movie reviews.
Given a review, based on their sentiment, you classify it into one of two categories: positive or negative.
Text: "I love this product! It's fantastic and works perfectly."
Positive
Text: "I'm really disappointed with the quality of the food."
Negative
Text: "This is the best day of my life!"
Positive
Text: "I can't stand the noise in this restaurant."
Negative
ONLY return the sentiment as output (without punctuation).
'''

In [47]:
subset = df.sample(5, random_state = 75)
print(subset, type(subset))

                                                    text     label
3406   After The Funeral was absolutely superb, and b...  positive
19464  I'm gonna tip the scales here a bit and say I ...  positive
14719  I run a group to stop comedian exploitation an...  negative
23125  I, like many horror fans, have been force fed ...  positive
15507  William Powell's final outing as Philo Vance o...  positive <class 'pandas.core.frame.DataFrame'>


In [34]:
user_data = subset['text'].to_string()

In [35]:
print(user_data, type(user_data))

32823    The central theme in this movie seems to be co...
16298    An excellent example of "cowboy noir", as it's...
28505    The ending made my heart jump up into my throa...
6689     Only the chosen ones will appreciate the quali...
26893    This is a really funny film, especially the se... <class 'str'>


In [53]:
def response(user_data):
    chat_completion = client.chat.completions.create(
        model = "gpt-4-turbo", 
        messages= (
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_data}
        )
    )
    
    return chat_completion.choices[0].message.content



In [54]:
subset['OpenAI_results'] = subset['text'].apply(response)
subset.head()

,text,label,OpenAI_results
3406,"After The Funeral was absolutely superb, and b...",positive,Positive
19464,I'm gonna tip the scales here a bit and say I ...,positive,Negative
14719,I run a group to stop comedian exploitation an...,negative,Negative
23125,"I, like many horror fans, have been force fed ...",positive,Positive
15507,William Powell's final outing as Philo Vance o...,positive,Positive


In [55]:
print(subset['text'].iloc[1])

I'm gonna tip the scales here a bit and say I enjoyed this. However, the cartoon is really only going to appeal to those who have very absurdist tendencies. It's definitely something that most people will not get, as is the nature of absurdism.<br /><br />the animation is horrible, but yes, that's the point. The main character is foul mouthed, violent, and stupid. no redeeming qualities whatsoever. his wife shrieks and wails, apparently just barely capable of the most basic communication skills. most of these stories completely lack any kind of point.<br /><br />but again, that's the point ;)<br /><br />If non sequiters, foul language, and complete and utter randomness are your thing, you're going to love this.<br /><br />It is really short, so I would probably rent instead of buying.
